# Parametrize Molecules with SMIRKS-based Force Field

## Basic Usage

SMIRKS is an extension of SMARTS language that enables users to define chemical substructures with certain patterns as well as to numerically tag the matching atoms. This allowed force field developers to introduce new parameters more easily by avoiding starting from defining new atom types. DMFF can deal with SMIRKS-based force field in XML format to create differentiable potential functions.

The usage of SMIRKS-based force field is generally the same as conventional force field based on atom-typing scheme, with the only difference such that we need an extra `rdkit.Chem.Mol` as input because the matching of SMIRKS pattern is powered by rdkit.

As an example, we will first load a N-methylacetamide molecule defined in a mol file.

In [ ]:
import jax.numpy as jnp
from rdkit import Chem
from dmff import Hamiltonian, NeighborList

mol = Chem.MolFromMolFile("C3H7NO.mol", removeHs=False) # hydrogens must be preserved

Then load force field parameters in xml format. Instuctions about how to write a SMIRKS-based force field XML file can be found in the Chapter 4 of the user's guide.

In [ ]:
h_smk = Hamiltonian("C3H7NO.xml", noOmmSys=True)

Note that the argument noOmmSys is set to False so that DMFF will not create an openmm system, as openmm does not support SMIRKS-based force field definitions.

Finally, we build an openmm topology and parametrize the molecule to create differentiable potential energy functions:

In [ ]:
top = h_smk.buildTopologyFromMol(mol)
potObj = h_smk.createPotential(top, rdmol=mol)
func = potObj.getPotentialFunc()

So far, we can utilize this dmff.Potential object to calculate energy and forces as we did in the previous sections.

In [ ]:
pos = jnp.array(mol.GetConformer().GetPositions()) / 10  # angstrom -> nm
box = jnp.eye(3, dtype=jnp.float32)
nblist = NeighborList(box, 1.0, potObj.meta["cov_map"])
nblist.allocate(pos)
pairs = nblist.pairs
energy = func(pos, box, pairs, h_smk.getParameters())
print(energy)

## Bond Charge Correction and Virtual Sites

This section mainly introduces how to use BCC and virtual sites in DMFF.

First, import required libraries:

In [1]:
import jax
import jax.numpy as jnp
from rdkit import Chem
from dmff import Hamiltonian, NeighborList

Load the molecule and SMIRKS-based force field file

In [4]:
mol = Chem.MolFromMolFile("clpy.mol", removeHs=False)
h_vsite = Hamiltonian("clpy_vsite.xml", noOmmSys=True)

TypeError: Hamiltonian.__init__() got an unexpected keyword argument 'noOmmSys'

In [7]:
from openmm import app
ff = app.ForceField("clpy_vsite.xml")
print(type(ff))
ff.createSystem()

<class 'openmm.app.forcefield.ForceField'>


TypeError: ForceField.createSystem() missing 1 required positional argument: 'topology'

BCC and virtual site parameters are parsed into `h_vsite.getParameters()['NonbondedForce']`

In [3]:
params = h_vsite.getParameters()
print(params['NonbondedForce'])

NameError: name 'h_vsite' is not defined

Build OpenMM topology and create DMFF potential

In [ ]:
top = h_vsite.buildTopologyFromMol(mol)
potObj = h_vsite.createPotential(top, rdmol=mol)

Add virtual site to RDKit Mol object

In [ ]:
mol_vsite = h_vsite.addVirtualSiteToMol(mol, h_vsite.getParameters())

Calculate energy, forces and parametric gradients

In [ ]:
pos_vsite = jnp.array(mol_vsite.GetConformer().GetPositions()) / 10  # angstrom -> nm
box = jnp.eye(3, dtype=jnp.float32)
nblist = NeighborList(box, 1.0, h_vsite.getCovalentMap())
nblist.allocate(pos_vsite)
pairs_vsite = nblist.pairs

nbfunc_vsite = jax.value_and_grad(
    potObj.dmff_potentials['NonbondedForce'], 
    argnums=-1, 
    allow_int=True # set to True since the type of virtual sites are speicified as integars
)
nbene_vsite, nbene_grad_vsite = nbfunc_vsite(pos_vsite, box, pairs_vsite, params)

Alternatively, we can also add coordinates of virtual sites by taking atomic positions matrix as an input.

In [ ]:
pos = jnp.array(mol.GetConformer().GetPositions()) / 10 # convert angstrom to nm
pos_vsite = h_vsite.addVirtualSiteCoords(pos, h_vsite.getParameters())
print(pos_vsite.shape)